# Explanation of the operation of this notebook

This notebook is designed to guide the user through a specific workflow, using code cells and Markdown to document and execute processes. Here the objectives, main steps and important considerations for their use are explained. Each section of the notebooks contains clear instructions and practical examples to facilitate understanding and learning.

In [0]:
from shapely import wkb
import geopandas as gpd

def spark_to_geopandas(df_spark, crs="EPSG:4326"):

    df = df_spark.toPandas()

    if isinstance(df['geometry'].iloc[0], bytes):
        df['geometry'] = df['geometry'].apply(wkb.loads)

    gdf = gpd.GeoDataFrame(df, geometry='geometry')

    if gdf.crs is None:
        gdf.set_crs(crs, inplace=True)
        
    return gdf

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4770404071401114>, line 1
----> 1 from shapely import wkb
      2 import geopandas as gpd
      4 def spark_to_geopandas(df_spark, crs="EPSG:4326"):                                                           #Definir función en utils.

File /databricks/python_shell/lib/dbruntime/autoreload/discoverability/hook.py:71, in AutoreloadDiscoverabilityHook._patched_import(self, name, *args, **kwargs)
     65 if not self._should_hint and (
     66     (module := sys.modules.get(absolute_name)) is not None and
     67     (fname := get_allowed_file_name_or_none(module)) is not None and
     68     (mtime := os.stat(fname).st_mtime) > self.last_mtime_by_modname.get(
     69         absolute_name, float("inf")) and not self._should_hint):
     70     self._should_hint = True
---> 71 module = self._original_builtins_import(name, *args,

In [0]:
import pytz

horario_colombia = pytz.timezone("America/Bogota")

In [0]:
from pyspark.sql.functions import col, avg, round

def calculate_bias_by_neighborhood(dataset):

    bias_by_neighborhood = dataset.groupBy("neighborhood").agg(
        avg("quantity_products").alias("avg_real"),
        avg("prediction").alias("avg_predicted")
    ).withColumn(
        "avg_bias", col("avg_predicted") - col("avg_real")
    ).withColumn(
        "bias_percentage", round((col("avg_bias") / col("avg_real")), 3)
    ).orderBy("neighborhood")

    return bias_by_neighborhood